In [ ]:
import numpy as np
from numpy import zeros

def RandomMiniBatches(X, Y, MiniBatchSize):

    m = X.shape[0]  
    miniBatches = [] 
   
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :]
    shuffled_Y = Y[permutation, :].reshape((m,1))   # Справедливо для формы uptpur

    num_minibatches = m // MiniBatchSize 
    for k in range(0, num_minibatches):
        miniBatch_X = shuffled_X[k * MiniBatchSize:(k + 1) * MiniBatchSize,:]
        miniBatch_Y = shuffled_Y[k * MiniBatchSize:(k + 1) * MiniBatchSize,:]
        miniBatch = (miniBatch_X, miniBatch_Y)
        miniBatches.append(miniBatch)
    
    #handeling last batch
    if m % MiniBatchSize != 0:
        # end = m - MiniBatchSize * m // MiniBatchSize
        miniBatch_X = shuffled_X[num_minibatches * MiniBatchSize:, :]
        miniBatch_Y = shuffled_Y[num_minibatches * MiniBatchSize:, :]

        miniBatch = (miniBatch_X, miniBatch_Y)
        miniBatches.append(miniBatch)
    
    return miniBatches 

In [ ]:
def update_params_with_momentum(params, grads, v, beta, learning_rate):
    
    # grads имеет параметры dw и db из предыдущих
    # параметров имеет параметры W и b, которые мы должны обновить
    for l in range(len(params) // 2 ):

        # ЗДЕСЬ МЫ ВЫЧИСЛЯЕМ СКОРОСТЬ
        v["dW" + str(l + 1)] = beta * v["dW" + str(l + 1)] + (1 - beta) * grads['dW' + str(l + 1)]
        v["db" + str(l + 1)] = beta * v["db" + str(l + 1)] + (1 - beta) * grads['db' + str(l + 1)]
        
        #updating params W and b
        params["W" + str(l + 1)] = params["W" + str(l + 1)] - learning_rate * v["dW" + str(l + 1)]
        params["b" + str(l + 1)] = params["b" + str(l + 1)] - learning_rate * v["db" + str(l + 1)]
    return params

In [ ]:
def model(X, Y, learning_rate, num_iter, hidden_size, keep_prob, optimizer):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    for i in range(1, num_iter):
        MiniBatches = RandomMiniBatches(X, Y, 64)   # Генерация случайных минипакетов
        p, q = MiniBatches[2]
        for MiniBatch in MiniBatches:               # Зацикливаем минипакеты

            (MiniBatch_X, MiniBatch_Y) = MiniBatch

            cache, A = model_forward(MiniBatch_X, params, L, keep_prob)             #ПРЯМОЕ РАСПРОСТРАНЕНИЕ
            cost = cost_f(A, MiniBatch_Y)                                           #СТОИМОСТЬ ФУНКЦИИ
            grads = backward(MiniBatch_X, MiniBatch_Y, params, cache, L, keep_prob)  #ОБРАТНОЕ РАСПРОСТРАНЕНИЕ
            params = update_params_with_momentum(params, grads, beta=0.9, learning_rate=learning_rate)
    return params